In [1]:
import requests,os,time
from bs4 import BeautifulSoup
from tqdm import tqdm

In [2]:
# fish_list.csvを開き、スクレイピング対象を取得する → search_listに格納
search_list = []

with open('fish_list.csv', mode='r', encoding='utf-8') as f:
    for row in f:
        columns = row.rstrip().split(',')
        search_list.append(columns)

print(search_list)

[['イシガキフグ', 'https://search.yahoo.co.jp/image/search?p=%E3%82%A4%E3%82%B7%E3%82%AC%E3%82%AD%E3%83%95%E3%82%B0&oq=&ei=UTF-8&b=', '「イシガキフグ」の画像検索結果'], ['カミソリウオ', 'https://search.yahoo.co.jp/image/search?p=%E3%82%AB%E3%83%9F%E3%82%BD%E3%83%AA%E3%82%A6%E3%82%AA&oq=&ei=UTF-8&b=', '「カミソリウオ」の画像検索結果'], ['ミノカサゴ', 'https://search.yahoo.co.jp/image/search?p=%E3%83%9F%E3%83%8E%E3%82%AB%E3%82%B5%E3%82%B4&oq=&ei=UTF-8&b=', '「ミノカサゴ」の画像検索結果'], ['']]


In [3]:
# Yahoo画像検索URLの取得関数
def get_page_urls(page_str):
    page_urls = []

    for n_page in range(1, 600, 20):
        page = page_str + '{}'.format(n_page)
        page_urls.append(page)

    return page_urls

#page_url = search_list[1][2]
#len(get_page_urls(page_url))

In [4]:
# 画像URL取得関数
def get_image(page_url, page_alt):
    r = requests.get(page_url)
    soup = BeautifulSoup(r.text, features='html.parser')
    
    img_tags = soup.find_all('img', attrs={'alt':page_alt})

    img_url = []

    for img_tag in img_tags:
        url = img_tag.get('src')
        img_url.append(url)
        
    return img_url

In [5]:
def download_image(url, file_path):
    r = requests.get(url, stream=True)
    
    if r.status_code == 200:
        with open(file_path, 'wb') as f:
            f.write(r.content)

In [6]:
for fish_name, page_str, page_alt in search_list:
    page_urls = get_page_urls(page_str)
    
    # 画像URLの取得
    img_urls = []
    for page_url in tqdm(page_urls): 
        img_url = get_image(page_url, page_alt)
        img_urls.extend(img_url)
        time.sleep(1)

    os.mkdir(fish_name)
    
    # 画像のダウンロード
    for i, img_url in tqdm(enumerate(img_urls)):
        path = fish_name + '/' + str(i).zfill(4) + '.jpg'
        download_image(url=img_urls[i], file_path=path)
        time.sleep(1)

100%|██████████| 30/30 [00:53<00:00,  1.80s/it]
600it [12:15,  1.23s/it]
100%|██████████| 30/30 [00:51<00:00,  1.73s/it]
600it [12:32,  1.25s/it]
100%|██████████| 30/30 [00:52<00:00,  1.74s/it]
600it [11:48,  1.18s/it]


ValueError: not enough values to unpack (expected 3, got 1)